In [1]:
import pandas as pd
import numpy as np
import holidays
import datetime
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore') 

import os
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
import random
import warnings
warnings.filterwarnings(action='ignore') 

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [2]:
building = pd.read_csv('building_info.csv')
building = building.replace('-', 'NaN')
building = building.astype({'태양광용량(kW)' : 'float', 'ESS저장용량(kWh)' : 'float', 'PCS용량(kW)' : 'float'})

In [3]:
train = pd.read_csv('train.csv')
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train['month'] = train['일시'].apply(lambda x : int(x[4:6]))
train['day'] = train['일시'].apply(lambda x : int(x[6:8]))
train['time'] = train['일시'].apply(lambda x : int(x[9:11]))
train['일시'] = train['일시'].str.replace(' ', '').astype('str')
train['일시'] = train['일시'] + str('00')
train['일시'] = pd.to_datetime(train['일시'])

In [4]:
# 일조 > 21시 ~ 5시
# 즉 일조는 6 ~ 20시까지 데이터 존재
# 일조는 0으로 결측치 처리하는게 맞다 
# 일단 강수량도 0으로 처리하니까 모델의 성능이 올랐다. 
# 일사는 21시 ~ 0시 > 0으로 처리하는게 맞다 + 나머지 시간은 평균 or bfill ffill이 가장 이상적(보니까 평균보다는 후자가 맞는듯?)

In [5]:
train.loc[train['time'].between(0, 5), '일조(hr)'] = train.loc[train['time'].between(0, 5), '일조(hr)'].fillna(0)
train.loc[train['time'].between(21, 23), '일조(hr)'] = train.loc[train['time'].between(21, 23), '일조(hr)'].fillna(0)
mean_radiation_by_time = train.groupby('time')['일조(hr)'].transform('mean')
train['일조(hr)'] = train['일조(hr)'].fillna(mean_radiation_by_time)

In [6]:
train.loc[train['time'].between(0, 5), '일사(MJ/m2)'] = train.loc[train['time'].between(0, 5), '일사(MJ/m2)'].fillna(0)
train.loc[train['time'].between(21, 23), '일사(MJ/m2)'] = train.loc[train['time'].between(21, 23), '일사(MJ/m2)'].fillna(0)
mean_radiation_by_time = train.groupby('time')['일사(MJ/m2)'].transform('mean')
train['일사(MJ/m2)'] = train['일사(MJ/m2)'].fillna(mean_radiation_by_time)

In [7]:
train['weekend'] = train['일시'].dt.weekday

In [8]:
# 평일 = 0, 공휴일 = 1
kr_holidays = holidays.Korea(years=train['일시'].dt.year.unique())
train['평일'] = train['일시'].apply(lambda x: 1 if x.weekday() >= 5 or x in kr_holidays else 0)

In [9]:
train['습도(%)'] = train['습도(%)'].interpolate()
train['풍속(m/s)'] = train['풍속(m/s)'].interpolate()
train.set_index('일시', inplace = True)

In [10]:
test = pd.read_csv('test.csv')
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
test['month'] = test['일시'].apply(lambda x : int(x[4:6]))
test['day'] = test['일시'].apply(lambda x : int(x[6:8]))
test['time'] = test['일시'].apply(lambda x : int(x[9:11]))
test['일시'] = test['일시'].str.replace(' ', '').astype('str')
test['일시'] = test['일시'] + str('00')
test['일시'] = pd.to_datetime(test['일시'])
# 평일 = 0, 공휴일 = 1
kr_holidays = holidays.Korea(years=test['일시'].dt.year.unique())
test['평일'] = test['일시'].apply(lambda x: 1 if x.weekday() >= 5 or x in kr_holidays else 0)
test['weekend'] = test['일시'].dt.weekday
test.set_index('일시', inplace = True)

In [11]:
train['강수량(mm)'].fillna(0, inplace = True)

In [12]:
# weather_x = train[['기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', 'month', 'day', 'time']]
# weather_y = train[['일조(hr)', '일사(MJ/m2)']]

In [13]:
# weather_test = test[['기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', 'month', 'day', 'time']]

In [127]:
# from sklearn.multioutput import MultiOutputRegressor

# base_model = lgb.LGBMRegressor()

# multioutput_regressor = MultiOutputRegressor(base_model)

# multioutput_regressor.fit(weather_x.values, weather_y.values)

# y_pred = multioutput_regressor.predict(weather_test.values)

In [128]:
# test[['일조(hr)', '일사(MJ/m2)']] = y_pred

In [129]:
# for i, j in test.iterrows():
#     if j['time'] in [0, 1, 2, 3, 4, 5, 21, 22, 23]:
#         test.at[i, '일조(hr)'] = 0
#         test.at[i, '일사(MJ/m2)'] = 0
#     if j['일조(hr)'] < 0:
#         test.at[i, '일조(hr)'] = 0
#     if j['일사(MJ/m2)'] < 0:
#         test.at[i, '일사(MJ/m2)'] = 0

In [14]:
all_table = pd.concat([train, test])

In [15]:
all_table = pd.merge(all_table, building, on = '건물번호', how = 'left')

In [16]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

encoder.fit(all_table['건물유형'])

all_table["건물유형"] = encoder.transform(all_table['건물유형'])

In [17]:
train = all_table.iloc[:train.shape[0], :]
test = all_table.iloc[train.shape[0] :, :]

In [18]:
train.head()

,num_date_time,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),month,day,time,weekend,평일,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1_20220601 00,1,18.6,0.0,0.9,42.0,0.0,0.0,1085.28,6,1,0,2,0,0,110634.0,39570.0,NaN,NaN,NaN
1,1_20220601 01,1,18.0,0.0,1.1,45.0,0.0,0.0,1047.36,6,1,1,2,0,0,110634.0,39570.0,NaN,NaN,NaN
2,1_20220601 02,1,17.7,0.0,1.5,45.0,0.0,0.0,974.88,6,1,2,2,0,0,110634.0,39570.0,NaN,NaN,NaN
3,1_20220601 03,1,16.7,0.0,1.4,48.0,0.0,0.0,953.76,6,1,3,2,0,0,110634.0,39570.0,NaN,NaN,NaN
4,1_20220601 04,1,18.4,0.0,2.8,43.0,0.0,0.0,986.40,6,1,4,2,0,0,110634.0,39570.0,NaN,NaN,NaN


In [19]:
test.head()

,num_date_time,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),month,day,time,weekend,평일,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
204000,1_20220825 00,1,23.5,0.0,2.2,72.0,NaN,NaN,NaN,8,25,0,3,0,0,110634.0,39570.0,NaN,NaN,NaN
204001,1_20220825 01,1,23.0,0.0,0.9,72.0,NaN,NaN,NaN,8,25,1,3,0,0,110634.0,39570.0,NaN,NaN,NaN
204002,1_20220825 02,1,22.7,0.0,1.5,75.0,NaN,NaN,NaN,8,25,2,3,0,0,110634.0,39570.0,NaN,NaN,NaN
204003,1_20220825 03,1,22.1,0.0,1.3,78.0,NaN,NaN,NaN,8,25,3,3,0,0,110634.0,39570.0,NaN,NaN,NaN
204004,1_20220825 04,1,21.8,0.0,1.0,77.0,NaN,NaN,NaN,8,25,4,3,0,0,110634.0,39570.0,NaN,NaN,NaN


In [20]:
train = train[['num_date_time', '건물번호', '건물유형', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
       '일조(hr)', '일사(MJ/m2)', 'month', 'day', 'time', '평일', 'weekend','연면적(m2)', '냉방면적(m2)', '전력소비량(kWh)']]

In [21]:
test = test[['num_date_time', '건물번호', '건물유형', '기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)',
       '일조(hr)', '일사(MJ/m2)', 'month', 'day', 'time', '평일', 'weekend','연면적(m2)', '냉방면적(m2)']]

In [22]:
sub = pd.read_csv('sample_submission.csv')

In [23]:
train

,num_date_time,건물번호,건물유형,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),month,day,time,평일,weekend,연면적(m2),냉방면적(m2),전력소비량(kWh)
0,1_20220601 00,1,0,18.6,0.0,0.9,42.0,0.0,0.000000,6,1,0,0,2,110634.00,39570.00,1085.28
1,1_20220601 01,1,0,18.0,0.0,1.1,45.0,0.0,0.000000,6,1,1,0,2,110634.00,39570.00,1047.36
2,1_20220601 02,1,0,17.7,0.0,1.5,45.0,0.0,0.000000,6,1,2,0,2,110634.00,39570.00,974.88
3,1_20220601 03,1,0,16.7,0.0,1.4,48.0,0.0,0.000000,6,1,3,0,2,110634.00,39570.00,953.76
4,1_20220601 04,1,0,18.4,0.0,2.8,43.0,0.0,0.000000,6,1,4,0,2,110634.00,39570.00,986.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,100_20220824 19,100,11,23.1,0.0,0.9,86.0,0.5,0.300450,8,24,19,0,2,57497.84,40035.23,881.04
203996,100_20220824 20,100,11,22.4,0.0,1.3,86.0,0.0,0.043287,8,24,20,0,2,57497.84,40035.23,798.96
203997,100_20220824 21,100,11,21.3,0.0,1.0,92.0,0.0,0.000000,8,24,21,0,2,57497.84,40035.23,825.12
203998,100_20220824 22,100,11,21.0,0.0,0.3,94.0,0.0,0.000000,8,24,22,0,2,57497.84,40035.23,640.08


In [229]:
train['건물유형'].value_counts() 

0     30600
1     16320
2     16320
4     16320
5     16320
6     16320
7     16320
8     16320
9     16320
10    16320
11    16320
3     10200
Name: 건물유형, dtype: int64

In [227]:
test['건물유형'].value_counts() 

0     2520
1     1344
2     1344
4     1344
5     1344
6     1344
7     1344
8     1344
9     1344
10    1344
11    1344
3      840
Name: 건물유형, dtype: int64

## 모델은 여기서 수정가능 

In [200]:
class run:
    def __init__(self, train, test, n, col_name1, col_name2, col_name3, window_size):
        self.train = train
        self.test = test
        self.n = n
        self.col_name1 = col_name1
        self.col_name2 = col_name2
        self.col_name3 = col_name3
        self.window_size = window_size
        
    def make_table(self):
        train = self.train
        n = self.n
        test = self.test
        train_table = train[train['건물번호'] == n]
        train_table = train_table.drop(['태양광용량(kW)', '일사(MJ/m2)', '일조(hr)','num_date_time', '발전량'], axis = 1)
        

        
        test_table = test[test['건물번호'] == n]
        test_table = test_table.drop(['일사(MJ/m2)', '일조(hr)', '발전량',
                         '태양광용량(kW)', 'num_date_time'], axis = 1)
        all_table = pd.concat([train_table, test_table])
        return all_table
    
#     def window_table(self):
#         all_table = self.make_table()
#         col_name1 = self.col_name1
#         col_name2 = self.col_name2
# #         col_name3 = self.col_name3
#         window_size = self.window_size
#         for i in range(1, window_size + 1):
#             col_window_name = col_name1 + "_Window" + str(i)
#             all_table[col_window_name] = all_table[col_name1].shift(i)
#         for i in range(1, window_size + 1):
#             col_window_name = col_name2 + "_Window" + str(i)
#             all_table[col_window_name] = all_table[col_name2].shift(i)
# #         for i in range(1, window_size + 1):
# #             col_window_name = col_name3 + "_Window" + str(i)
# #             all_table[col_window_name] = all_table[col_name3].shift(i)
#         train = all_table.iloc[: 2040, : ]
#         test = all_table.iloc[len(train):, :]
#         train.dropna(inplace = True)
#         test = test.drop('전력소비량(kWh)', axis =1)
#         return train, test

    
    def window_table(self):
        all_table = self.make_table()
        col_name1 = self.col_name1
        col_name2 = self.col_name2
        col_name3 = self.col_name3
        window_size = self.window_size
        for i in range(1, window_size + 1):
            col_window_name = col_name1 + "_Window" + str(i)
            all_table[col_window_name] = all_table[col_name1].shift(i)
        for i in range(1, window_size + 1):
            col_window_name = col_name2 + "_Window" + str(i)
            all_table[col_window_name] = all_table[col_name2].shift(i)
        for i in range(1, window_size + 1):
            col_window_name = col_name3 + "_Window" + str(i)
            all_table[col_window_name] = all_table[col_name3].shift(i)    

        for i in range(1, window_size + 1):
            col_window_diff_name = col_name1 + "_diff_Window" + str(i)
            all_table[col_window_diff_name] = all_table[col_name1].diff(i)    
        for i in range(1, window_size + 1):
            col_window_diff_name = col_name2 + "_diff_Window" + str(i)
            all_table[col_window_diff_name] = all_table[col_name2].diff(i)    
        
        train = all_table.iloc[: 2040, : ]
        test = all_table.iloc[len(train):, :]
        train.dropna(inplace = True)
        train_x = train.drop('전력소비량(kWh)', axis =1)
        train_y = train['전력소비량(kWh)']
        test = test.drop('전력소비량(kWh)', axis =1)
        
        return train_x, train_y, test
        
    
    def model(self):
        train_x, train_y, test = self.window_table()
        model = lgb.LGBMRegressor()
        model.fit(train_x, train_y)
        preds = model.predict(test)
        return preds

In [24]:
class run:
    def __init__(self, train, test, n, col_name1, col_name2, col_name3, window_size):
        self.train = train
        self.test = test
        self.n = n
        self.col_name1 = col_name1
        self.col_name2 = col_name2
        self.col_name3 = col_name3
        self.window_size = window_size
        
    def make_table(self):
        train = self.train
        n = self.n
        test = self.test
        train_table = train[train['건물번호'] == n]
        train_table = train_table.drop(['일사(MJ/m2)', '일조(hr)','num_date_time'], axis = 1)
        

        
        test_table = test[test['건물번호'] == n]
        test_table = test_table.drop(['일사(MJ/m2)', '일조(hr)', 'num_date_time'], axis = 1)
        all_table = pd.concat([train_table, test_table])
        return all_table
    
#     def window_table(self):
#         all_table = self.make_table()
#         col_name1 = self.col_name1
#         col_name2 = self.col_name2
# #         col_name3 = self.col_name3
#         window_size = self.window_size
#         for i in range(1, window_size + 1):
#             col_window_name = col_name1 + "_Window" + str(i)
#             all_table[col_window_name] = all_table[col_name1].shift(i)
#         for i in range(1, window_size + 1):
#             col_window_name = col_name2 + "_Window" + str(i)
#             all_table[col_window_name] = all_table[col_name2].shift(i)
# #         for i in range(1, window_size + 1):
# #             col_window_name = col_name3 + "_Window" + str(i)
# #             all_table[col_window_name] = all_table[col_name3].shift(i)
#         train = all_table.iloc[: 2040, : ]
#         test = all_table.iloc[len(train):, :]
#         train.dropna(inplace = True)
#         test = test.drop('전력소비량(kWh)', axis =1)
#         return train, test

    
    def window_table(self):
        all_table = self.make_table()
        col_name1 = self.col_name1
        col_name2 = self.col_name2
        col_name3 = self.col_name3
        window_size = self.window_size
        for i in range(1, window_size + 1):
            col_window_name = col_name1 + "_Window" + str(i)
            all_table[col_window_name] = all_table[col_name1].shift(i)
        for i in range(1, window_size + 1):
            col_window_name = col_name2 + "_Window" + str(i)
            all_table[col_window_name] = all_table[col_name2].shift(i)
        for i in range(1, window_size + 1):
            col_window_name = col_name3 + "_Window" + str(i)
            all_table[col_window_name] = all_table[col_name3].shift(i)    

#         for i in range(1, window_size + 1):
#             col_window_diff_name = col_name1 + "_diff_Window" + str(i)
#             all_table[col_window_diff_name] = all_table[col_name1].diff(i)    
#         for i in range(1, window_size + 1):
#             col_window_diff_name = col_name2 + "_diff_Window" + str(i)
#             all_table[col_window_diff_name] = all_table[col_name2].diff(i)    
        return all_table
    
    def last_table(self):
        all_table = self.window_table()
        last = []
        for i in range(1, 101):
            output = run(train, test, i, '기온(C)', '습도(%)', '강수량(mm)', 96).window_table()
            last.append(output)
        last = np.array(last)
        last = last.reshape(-1, 302)
        last = pd.DataFrame(last, columns = all_table.columns)
        return last
    
    def split_table(self):
        last = self.last_table()
        n = self.n
        last = last[last['건물번호'] == n]
        train = last.iloc[:2040, :]
        test = last.iloc[len(train) :, :]
        test.drop('전력소비량(kWh)', axis = 1, inplace = True)
        train.dropna(inplace = True)
        return train, test    
    
#     def real_table(self, bul_type):
#         train, test = self.split_table()
#         self.bul_type = bul_type
#         train_table = train[train['건물번호'] == bul_type]
        
        
        
    
    def model(self):
        train_x, train_y, test = self.window_table()
        model = lgb.LGBMRegressor()
        model.fit(train_x, train_y)
        preds = model.predict(test)
        return preds

In [35]:
a = run(train, test, 1, '기온(C)', '습도(%)', '강수량(mm)', 96).split_table()[0]

In [43]:
b = run(train, test, 2, '기온(C)', '습도(%)', '강수량(mm)', 96).split_table()[0]

In [50]:
sibar = [a, b]
sibar = np.array(sibar)
sibar = sibar.reshape(-1, 302)
pd.DataFrame(sibar)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301
0,1.0,0.0,24.8,0.0,2.1,60.0,6.0,5.0,0.0,1.0,6.0,110634.00,39570.0,963.36,25.6,26.1,26.7,27.3,28.4,29.4,29.2,29.1,29.0,28.7,28.4,28.2,27.8,26.1,24.6,23.2,21.2,20.4,20.9,21.0,21.4,21.5,21.6,22.3,22.7,23.2,23.9,24.3,25.9,27.9,29.0,30.9,32.3,31.2,29.8,28.3,26.7,24.9,23.1,21.0,19.6,19.1,19.2,19.1,19.1,19.2,19.0,18.9,19.2,19.7,20.4,21.6,23.3,24.9,25.6,26.4,26.7,26.3,25.9,25.4,24.2,22.5,21.0,19.0,18.4,17.8,17.6,17.2,17.2,17.2,17.6,18.1,18.9,19.6,20.7,22.2,24.3,26.3,27.1,27.6,28.3,28.4,28.3,27.5,26.3,24.8,...,48.0,45.0,45.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,24.5,0.0,1.7,58.0,6.0,5.0,1.0,1.0,6.0,110634.00,39570.0,872.16,24.8,25.6,26.1,26.7,27.3,28.4,29.4,29.2,29.1,29.0,28.7,28.4,28.2,27.8,26.1,24.6,23.2,21.2,20.4,20.9,21.0,21.4,21.5,21.6,22.3,22.7,23.2,23.9,24.3,25.9,27.9,29.0,30.9,32.3,31.2,29.8,28.3,26.7,24.9,23.1,21.0,19.6,19.1,19.2,19.1,19.1,19.2,19.0,18.9,19.2,19.7,20.4,21.6,23.3,24.9,25.6,26.4,26.7,26.3,25.9,25.4,24.2,22.5,21.0,19.0,18.4,17.8,17.6,17.2,17.2,17.2,17.6,18.1,18.9,19.6,20.7,22.2,24.3,26.3,27.1,27.6,28.3,28.4,28.3,27.5,26.3,...,43.0,48.0,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,23.8,0.0,1.2,60.0,6.0,5.0,2.0,1.0,6.0,110634.00,39570.0,798.24,24.5,24.8,25.6,26.1,26.7,27.3,28.4,29.4,29.2,29.1,29.0,28.7,28.4,28.2,27.8,26.1,24.6,23.2,21.2,20.4,20.9,21.0,21.4,21.5,21.6,22.3,22.7,23.2,23.9,24.3,25.9,27.9,29.0,30.9,32.3,31.2,29.8,28.3,26.7,24.9,23.1,21.0,19.6,19.1,19.2,19.1,19.1,19.2,19.0,18.9,19.2,19.7,20.4,21.6,23.3,24.9,25.6,26.4,26.7,26.3,25.9,25.4,24.2,22.5,21.0,19.0,18.4,17.8,17.6,17.2,17.2,17.2,17.6,18.1,18.9,19.6,20.7,22.2,24.3,26.3,27.1,27.6,28.3,28.4,28.3,27.5,...,46.0,43.0,48.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,23.5,0.0,1.6,59.0,6.0,5.0,3.0,1.0,6.0,110634.00,39570.0,780.00,23.8,24.5,24.8,25.6,26.1,26.7,27.3,28.4,29.4,29.2,29.1,29.0,28.7,28.4,28.2,27.8,26.1,24.6,23.2,21.2,20.4,20.9,21.0,21.4,21.5,21.6,22.3,22.7,23.2,23.9,24.3,25.9,27.9,29.0,30.9,32.3,31.2,29.8,28.3,26.7,24.9,23.1,21.0,19.6,19.1,19.2,19.1,19.1,19.2,19.0,18.9,19.2,19.7,20.4,21.6,23.3,24.9,25.6,26.4,26.7,26.3,25.9,25.4,24.2,22.5,21.0,19.0,18.4,17.8,17.6,17.2,17.2,17.2,17.6,18.1,18.9,19.6,20.7,22.2,24.3,26.3,27.1,27.6,28.3,28.4,28.3,...,50.0,46.0,43.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [55]:
train_table = []
test_table = []

for i in range(1, 101):
    train_output = run(train, test, i, '기온(C)', '습도(%)', '강수량(mm)', 96).split_table()[0]
    train_table.append(train_output)
    test_output = run(train, test, i, '기온(C)', '습도(%)', '강수량(mm)', 96).split_table()[1]
    test_table.append(test_output)

    
train_table = np.array(train_table)
train_table = train_table.reshape(-1, 302)
train_table = pd.DataFrame(train_table, columns = a.columns)    
test_table = np.array(test_table)
test_table = test_table.reshape(-1, 302)
test_table = pd.DataFrame(test_table, columns = a.columns)

ValueError: cannot reshape array of size 5056800 into shape (302)

In [53]:
train_table

,건물번호,건물유형,기온(C),강수량(mm),풍속(m/s),습도(%),month,day,time,평일,weekend,연면적(m2),냉방면적(m2),전력소비량(kWh),기온(C)_Window1,기온(C)_Window2,기온(C)_Window3,기온(C)_Window4,기온(C)_Window5,기온(C)_Window6,기온(C)_Window7,기온(C)_Window8,기온(C)_Window9,기온(C)_Window10,기온(C)_Window11,기온(C)_Window12,기온(C)_Window13,기온(C)_Window14,기온(C)_Window15,기온(C)_Window16,기온(C)_Window17,기온(C)_Window18,기온(C)_Window19,기온(C)_Window20,기온(C)_Window21,기온(C)_Window22,기온(C)_Window23,기온(C)_Window24,기온(C)_Window25,기온(C)_Window26,기온(C)_Window27,기온(C)_Window28,기온(C)_Window29,기온(C)_Window30,기온(C)_Window31,기온(C)_Window32,기온(C)_Window33,기온(C)_Window34,기온(C)_Window35,기온(C)_Window36,기온(C)_Window37,기온(C)_Window38,기온(C)_Window39,기온(C)_Window40,기온(C)_Window41,기온(C)_Window42,기온(C)_Window43,기온(C)_Window44,기온(C)_Window45,기온(C)_Window46,기온(C)_Window47,기온(C)_Window48,기온(C)_Window49,기온(C)_Window50,기온(C)_Window51,기온(C)_Window52,기온(C)_Window53,기온(C)_Window54,기온(C)_Window55,기온(C)_Window56,기온(C)_Window57,기온(C)_Window58,기온(C)_Window59,기온(C)_Window60,기온(C)_Window61,기온(C)_Window62,기온(C)_Window63,기온(C)_Window64,기온(C)_Window65,기온(C)_Window66,기온(C)_Window67,기온(C)_Window68,기온(C)_Window69,기온(C)_Window70,기온(C)_Window71,기온(C)_Window72,기온(C)_Window73,기온(C)_Window74,기온(C)_Window75,기온(C)_Window76,기온(C)_Window77,기온(C)_Window78,기온(C)_Window79,기온(C)_Window80,기온(C)_Window81,기온(C)_Window82,기온(C)_Window83,기온(C)_Window84,기온(C)_Window85,기온(C)_Window86,...,습도(%)_Window93,습도(%)_Window94,습도(%)_Window95,습도(%)_Window96,강수량(mm)_Window1,강수량(mm)_Window2,강수량(mm)_Window3,강수량(mm)_Window4,강수량(mm)_Window5,강수량(mm)_Window6,강수량(mm)_Window7,강수량(mm)_Window8,강수량(mm)_Window9,강수량(mm)_Window10,강수량(mm)_Window11,강수량(mm)_Window12,강수량(mm)_Window13,강수량(mm)_Window14,강수량(mm)_Window15,강수량(mm)_Window16,강수량(mm)_Window17,강수량(mm)_Window18,강수량(mm)_Window19,강수량(mm)_Window20,강수량(mm)_Window21,강수량(mm)_Window22,강수량(mm)_Window23,강수량(mm)_Window24,강수량(mm)_Window25,강수량(mm)_Window26,강수량(mm)_Window27,강수량(mm)_Window28,강수량(mm)_Window29,강수량(mm)_Window30,강수량(mm)_Window31,강수량(mm)_Window32,강수량(mm)_Window33,강수량(mm)_Window34,강수량(mm)_Window35,강수량(mm)_Window36,강수량(mm)_Window37,강수량(mm)_Window38,강수량(mm)_Window39,강수량(mm)_Window40,강수량(mm)_Window41,강수량(mm)_Window42,강수량(mm)_Window43,강수량(mm)_Window44,강수량(mm)_Window45,강수량(mm)_Window46,강수량(mm)_Window47,강수량(mm)_Window48,강수량(mm)_Window49,강수량(mm)_Window50,강수량(mm)_Window51,강수량(mm)_Window52,강수량(mm)_Window53,강수량(mm)_Window54,강수량(mm)_Window55,강수량(mm)_Window56,강수량(mm)_Window57,강수량(mm)_Window58,강수량(mm)_Window59,강수량(mm)_Window60,강수량(mm)_Window61,강수량(mm)_Window62,강수량(mm)_Window63,강수량(mm)_Window64,강수량(mm)_Window65,강수량(mm)_Window66,강수량(mm)_Window67,강수량(mm)_Window68,강수량(mm)_Window69,강수량(mm)_Window70,강수량(mm)_Window71,강수량(mm)_Window72,강수량(mm)_Window73,강수량(mm)_Window74,강수량(mm)_Window75,강수량(mm)_Window76,강수량(mm)_Window77,강수량(mm)_Window78,강수량(mm)_Window79,강수량(mm)_Window80,강수량(mm)_Window81,강수량(mm)_Window82,강수량(mm)_Window83,강수량(mm)_Window84,강수량(mm)_Window85,강수량(mm)_Window86,강수량(mm)_Window87,강수량(mm)_Window88,강수량(mm)_Window89,강수량(mm)_Window90,강수량(mm)_Window91,강수량(mm)_Window92,강수량(mm)_Window93,강수량(mm)_Window94,강수량(mm)_Window95,강수량(mm)_Window96
0,1.0,0.0,24.8,0.0,2.1,60.0,6.0,5.0,0.0,1.0,6.0,110634.0,39570.0,963.36,25.6,26.1,26.7,27.3,28.4,29.4,29.2,29.1,29.0,28.7,28.4,28.2,27.8,26.1,24.6,23.2,21.2,20.4,20.9,21.0,21.4,21.5,21.6,22.3,22.7,23.2,23.9,24.3,25.9,27.9,29.0,30.9,32.3,31.2,29.8,28.3,26.7,24.9,23.1,21.0,19.6,19.1,19.2,19.1,19.1,19.2,19.0,18.9,19.2,19.7,20.4,21.6,23.3,24.9,25.6,26.4,26.7,26.3,25.9,25.4,24.2,22.5,21.0,19.0,18.4,17.8,17.6,17.2,17.2,17.2,17.6,18.1,18.9,19.6,20.7,22.2,24.3,26.3,27.1,27.6,28.3,28.4,28.3,27.5,26.3,24.8,...,48.0,45.0,45.0,42.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,24.5,0.0,1.7,58.0,6

In [288]:
# result = []
# for i in range(1, 101):
#     output = run(train, test, i, '기온(C)', '습도(%)', '강수량(mm)', 96).model()
#     result.append(output)

In [211]:
result = []
for i in range(12):
    output = run(train, test, i, '기온(C)', '습도(%)', '강수량(mm)', 96).model()
    result.append(output)

In [212]:
result = np.array(result)
result = result.reshape(-1, 1)
result.shape

(12, 1)

In [213]:
sub['answer'] = result
sub

ValueError: Length of values (12) does not match length of index (16800)

In [257]:
sub.to_csv('0812_강수_마지막.csv', index = False)

In [259]:
conda install ruptures

ValueError: The python kernel does not appear to be a conda environment.  Please use ``%pip install`` instead.